In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import image_dataset_from_directory
import tensorflow_datasets as tfds

import numpy as np
import os
import PIL
import PIL.Image
import glob

import cv2

import matplotlib.pyplot as plt
from matplotlib import gridspec

ModuleNotFoundError: No module named 'tensorflow'

In [2]:
physical_devices = tf.config.list_physical_devices('GPU')
print("Num GPUs:", len(physical_devices))

Num GPUs: 1


2022-09-12 16:18:03.769788: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-12 16:18:03.799119: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-12 16:18:03.799254: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [6]:
train_dataset = image_dataset_from_directory(
'imagenette/train',
image_size=(213, 160),
batch_size=32)

val_dataset = image_dataset_from_directory(
'imagenette/val',
image_size=(213,160),
batch_size=32)

Found 9469 files belonging to 10 classes.
Found 3925 files belonging to 10 classes.


In [7]:
conv_base = keras.applications.vgg16.VGG16(
weights="imagenet",
input_shape=(213, 160, 3),
include_top=False)
conv_base.trainable = False

58889256/58889256 [==============================] - 3s 0us/step


In [11]:
data_augmentation = keras.Sequential(
[
layers.RandomFlip("horizontal"),
layers.RandomRotation(0.1),
layers.RandomZoom(0.2),
]
)

In [12]:
inputs = keras.Input(shape=(213, 160, 3))
x = data_augmentation(inputs)
x = keras.applications.vgg16.preprocess_input(x)
x = conv_base(x)
x = layers.Flatten()(x)
x = layers.Flatten()(x)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(10, activation="softmax")(x)
model = keras.Model(inputs, outputs)
model.compile(loss="SparseCategoricalCrossentropy",
optimizer="rmsprop",
metrics=["accuracy"])

In [13]:
callbacks = [
keras.callbacks.ModelCheckpoint(
filepath="feature_extraction_with_data_augmentation.keras",
save_best_only=True,
monitor="val_loss")
]

In [14]:
history = model.fit(
train_dataset,
epochs=50,
validation_data=val_dataset,
callbacks=callbacks)

Epoch 1/50
296/296 [==============================] - 16s 52ms/step - loss: 21.4250 - accuracy: 0.8327 - val_loss: 10.5146 - val_accuracy: 0.9287
Epoch 2/50
296/296 [==============================] - 15s 51ms/step - loss: 14.6794 - accuracy: 0.8937 - val_loss: 9.4628 - val_accuracy: 0.9338
Epoch 3/50
296/296 [==============================] - 15s 50ms/step - loss: 12.2211 - accuracy: 0.9067 - val_loss: 8.6014 - val_accuracy: 0.9419
Epoch 4/50
296/296 [==============================] - 15s 50ms/step - loss: 9.7589 - accuracy: 0.9183 - val_loss: 8.6847 - val_accuracy: 0.9401
Epoch 5/50
296/296 [==============================] - 15s 50ms/step - loss: 9.0612 - accuracy: 0.9216 - val_loss: 10.6266 - val_accuracy: 0.9366
Epoch 6/50
296/296 [==============================] - 15s 50ms/step - loss: 7.4856 - accuracy: 0.9302 - val_loss: 8.2987 - val_accuracy: 0.9409
Epoch 7/50
296/296 [==============================] - 15s 50ms/step - loss: 6.9224 - accuracy: 0.9318 - val_loss: 8.7881 - val_accu

In [1]:
# Multiply Function

def multiply(img):
    # Get the size of the image
    h = img.shape[0]
    w = img.shape[1]
    d = img.shape[2]

    # Check if image is normalized between 0 - 1.
    if (max(img[0][0]) > 1):
        # Convert to float & normalize
        img = img.astype("float64") / 255

    # Apply the Filter
    for y in range(0,h):
        #Search each pixel in the row. x = pixel
        for x in range(0,w):
            #Search each RGB value.
            for z in range(0,d):
                # Apply Filter to seperate image. 
                img[y,x,z] = img[y,x,z] * img[y,x,z]
    
    # return Multiply image
    return img

# Screen Function

def screen(img):
    # Get the size of the image
    h = img.shape[0]
    w = img.shape[1]
    d = img.shape[2]

    # Check if image is normalized between 0 - 1.
    if (max(img[0][0]) > 1):
        # Convert to float & normalize
        img = img.astype("float64") / 255

    # Apply the Filter
    for y in range(0,h):
        #Search each pixel in the row. x = pixel
        for x in range(0,w):
            #Search each RGB value.
            for z in range(0,d):
                # Apply Filter to seperate image. 
                img[y,x,z] = 1 - (1 - img[y,x,z]) * (1 - img[y,x,z])
    
    # return Multiply image
    return img
